<a href="https://colab.research.google.com/github/ynaowusu/protein-folding-quantum-algorithms/blob/main/proteinfolding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install qiskit #this is just me intalling qiskit into our notebook
!pip install matplotlib plotly #since it says we need a 3d structure to simulate the lattice and any other 3d elements
!pip install numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 8.6 MB/s eta 0:00:00


In [4]:
import numpy as np
import pytest
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from typing import List, Dict, Tuple
from qiskit.quantum_info import SparsePauliOp


In [5]:
"""
        proteins fold in 3D space and here we're working with a simplified lattice model w tetrahedral directions. There are two kinds of alternating sites in the protein chain: 'A' and 'B'.
        every turn corresponds to a direction in 3D space either x,y or z
        The vectors  [1, 1, 1]) are unit steps in different directions based on the protein structure model.
"""

class ProtienFoldVector:
  def __init_(self):
    self.lattice_vector = {
        "A":{
            0:[(1,1,1)], #|00> This is for the first round of qubits reprsents x,y,z
            1:[(1,-1,-1)], #|10>
            2:[(-1,1,-1)], #|01>
            3:[(-1,-1,-1)] #|11>
        },
        "B":{
            0:[(1,1,-1)], #|00>
            1:[(1,-1,1)], #|10>
            2:[(-1,1,1)], #|01>
            3:[(-1,-1,-1)] #11>
        }





    }

    pass


    #vectors go here


In [6]:

"""
  - Sparse (4-qubit per turn) one-hot encoding on the alternating A/B diamond lattice.
  - Qiskit helpers to allocate and prepare the configuration-qubit register.
  - Decoding measured bitstrings into 3D backbone turn vectors.

Usage:
  from hamiltonian import (
      create_configuration_circuit,
      decode_measurement,
      SparseDiamondEncoder
  )

  qc, cfg, meas = create_configuration_circuit(N)
  # ... add variational ansatz, measure, run on backend ...
  counts = backend.run(qc).result().get_counts()
  turns = decode_measurement(counts, N)
"""


class SparseDiamondEncoder:
    """Encodes/decodes backbone turns on a diamond (tetrahedral) lattice."""

    # sublattice A basis (even sites)
    _A: Dict[int,np.ndarray] = {
        0: np.array([+1,+1,+1]),
        1: np.array([+1,-1,-1]),
        2: np.array([-1,+1,-1]),
        3: np.array([-1,-1,+1]),
    }
    # sublattice B basis (odd sites)
    _B: Dict[int,np.ndarray] = {
        0: np.array([+1,+1,-1]),
        1: np.array([+1,-1,+1]),
        2: np.array([-1,+1,+1]),
        3: np.array([-1,-1,-1]),
    }

    @staticmethod
    def site_parity(i) :
        """Return 'A' if turn index i is even (sublattice A), else 'B'."""
        return 'A' if i % 2 == 0 else 'B'

    @classmethod
    def block_to_vec(cls, block, i) :
        """
        Decode one-hot 4-bit string (e.g. '1000') at turn index i into a 3D step vector.
        Raises ValueError if block is not one-hot of length 4.
        """
        if len(block) != 4 or block.count('1') != 1:
            raise ValueError(f"Invalid block {block}, must be one-hot length 4.")
        axis = ['1000','0100','0010','0001'].index(block)
        if cls.site_parity(i) == 'A':
            table = cls._A
        else:
            table = cls._B
        return table[axis]
    @classmethod
    def bitstring_to_turns(cls, bitstr, N) :
        """
        Convert full measured bitstring (length=4*(N-3)) into list of N-1 turn vectors:
        - t1 = (+1,+1,+1)
        - t2 = (+1,-1,-1)
        - t3..t_{N-1} from blocks of 4 bits
        """
        expected = 4*(N-3)
        if len(bitstr) != expected:
            raise ValueError(f"Expected bitstring length {expected}, got {len(bitstr)}")
        turns: List[np.ndarray] = []
        # fixed first two turns
        turns.append(cls._A[0])  # t1
        turns.append(cls._A[1])  # t2
        # decode remaining
        for k, start in enumerate(range(0, expected, 4), start=3):
            block = bitstr[start:start+4]
            turns.append(cls.block_to_vec(block, k))
        return turns


def create_configuration_circuit(N):
    """
    Allocate 4*(N-3) config qubits in a register, apply Hadamard to each,
    and attach classical bits for measurement.
    Returns (qc, cfg, meas).
    """
    n_cfg = 4*(N-3)
    cfg = QuantumRegister(n_cfg, 'cfg')
    meas = ClassicalRegister(n_cfg, 'meas')
    qc = QuantumCircuit(cfg, meas)
    qc.h(cfg)
    return qc, cfg, meas


def decode_measurement(counts, N):
    """
    From measurement counts, pick the most frequent bitstring and decode into turn vectors.
    """
    bitstr = max(counts, key=counts.get)
    return SparseDiamondEncoder.bitstring_to_turns(bitstr, N)

In [8]:
"""
Hamiltonian  defines the cost Hamiltonian terms for a protein fold on a diamond lattice:

1. Growth Constraint (H_gc): penalises consecutive backtracking bonds.
2. Overlap Penalty (H_ov): penalises any pair of beads occupying the same site.
3. Contact Reward (H_ct): rewards non-bonded nearest neighbours.
4. Chirality Constraint (H_ch): enforces correct handedness per sublattice.
5. Interaction Stub (H_in): placeholder for shell-based interaction qubits.

Each method returns a classical float energy for the decoded turn list.
"""

def T(t_i: np.ndarray, t_j: np.ndarray) -> int:
    """
    Indicator: 1 if t_i and t_j lie on same lattice axis or opposite, else 0.
    """
    return int(np.array_equal(t_i, t_j) or np.array_equal(t_i, -t_j))


class Hamiltonian:
    def __init__(
        self,
        turns: List[np.ndarray],
        back_penalty_weight: float = 30.0,
        overlap_weight: float = 50.0,
        contact_reward: float = 10.0,
        chirality_weight: float = 40.0,
        interaction_weights=None,
    ):
        """
        Parameters:
        -----------
        turns : List of N-1 3D step vectors (np.ndarray).
        back_penalty_weight : λ_back for growth constraint.
        overlap_weight : λ_ov for overlap penalty.
        contact_reward : ε for contact reward (>0).
        chirality_weight : λ_chir for chirality penalty.
        """
        self.turns = turns
        self.N = len(turns) + 1
        self.λ_back = back_penalty_weight
        self.λ_ov = overlap_weight
        self.ε = contact_reward
        self.λ_chir = chirality_weight
        self.interaction_weights = interaction_weights or {}
    def growth_constraint(self):
        """
        H_gc = λ_back * Σ_{i=3..N-1} T(t_i, t_{i+1}).
        """
        H_gc = 0.0
        for i in range(2, self.N - 2):
            H_gc += self.λ_back * T(self.turns[i], self.turns[i + 1])
        return H_gc

    def overlap_penalty(self):
        """
        H_ov = λ_ov * Σ_{i2 and nearest neighbours} 1,
        where nearest neighbours if Euclidean distance == 1.
        """
        pos = [(0, 0, 0)]
        for t in self.turns:
            prev = np.array(pos[-1])
            pos.append(tuple(prev + t))
        H_ct = 0.0
        # threshold for nearest neighbour on diamond lattice
        nn_dist = np.sqrt(3)
        tol = 1e-6
        for i in range(self.N):
            # |i-j|>2
            for j in range(i + 3, self.N):
                d = np.linalg.norm(np.array(pos[i]) - np.array(pos[j]))
                if abs(d - nn_dist) < tol:
                    H_ct += -self.ε
        return H_ct
    def chirality_constraint(self):
        """
        H_ch = λ_chir * Σ_{i=2..N-2} indicator of wrong handedness;
        uses scalar triple product and sublattice parity.
        """
        H_ch = 0.0
        for i in range(2, self.N - 2):
            v_prev = self.turns[i - 1]
            v_curr = self.turns[i]
            v_side = self.turns[i + 1]
            triple = np.dot(np.cross(v_prev, v_curr), v_side)
            parity = (1 - (-1) ** i) // 2
            # expect triple>0 on A (parity=0), <0 on B (parity=1)
            if not ((parity == 0 and triple > 0) or (parity == 1 and triple < 0)):
                H_ch += self.λ_chir
        return H_ch


    def interaction_constraint(self):

        H_in = 0.0
        # build bead positions
        pos = [(0, 0, 0)]
        for t in self.turns:
            prev = np.array(pos[-1])
            pos.append(tuple(prev + t))
        tol = 1e-6
        for shell, weight in self.interaction_weights.items():
            for i in range(self.N):
                for j in range(i + 3, self.N):
                    d = np.linalg.norm(np.array(pos[i]) - np.array(pos[j]))
                    if abs(d - shell) < tol:
                        H_in += weight
        return H_in
    def total_energy(self):
        """
        Total classical energy: H_gc + H_ov + H_ct + H_ch + H_in
        """
        return (
            self.growth_constraint()
            + self.overlap_penalty()
            + self.contact_reward()
            + self.chirality_constraint()
            + self.interaction_constraint()
        )


7


In [9]:
from qiskit import QuantumCircuit

class CVARVQE:

    def __init__(self, hamiltonian, alpha = 0.1):
        # self is just the initializer
        # the hamiltonian calculates the amount of energy it takes to form any given protein configuration.
        # this is important because the optimizer needs a way to evaluate how "good" or "bad" a quantum circuit’s result is.
        # alpha = 0.1 means CVaR will only average the best 10% of outputs.
        self.hamiltonian = hamiltonian  # Save the energy calculator
        self.alpha = alpha              # Only use the best 10% of folds
        self.n_qubits = hamiltonian.total_qubits  # Know how many qubits to use in the circuit

    def create_ansatz(self, params):
        qc = QuantumCircuit(self.n_qubits)
        # This creates a quantum circuit with the number of qubits based on the protein

        # Step 1: Put each qubit into superposition using Hadamard gates
        for i in range(self.n_qubits):
            qc.h(i)  # Hadamard gate turns each qubit into a mix of 0 and 1

        # Step 2: Add a rotation gate to each qubit (this is how we "teach" the circuit how to fold the protein)
        param_1 = 0  # Keeps track of which parameter we're using from the list

        for i in range(self.n_qubits):
            qc.ry(params[param_1], i)  # apply RY (Y-axis) rotation to qubit i using params[param_1]
            param_1 += 1  # Move to the next angle in the list

        for i in range(self.n_qubits - 1): #what this is is that it's making a CNOT gate to make an entanglement between two qubits
          qc.cx(i, i+ 1 ) #this entanglement lets qubits share information which is important because in proteins, one fold affects nearby folds.

        for i in range(self.n_qubits):
          qc.ry(params[param_1], i) #this is a repeat of the other line of code that just rotates the qubit again after the cnot entangled gate is applied
          param_1 += 1 #the reason it's roated again is so that the portien can fold in more complex ways.


        return qc  # Return the final circuit
    def evaluate_energy(self, params, n_shots = 2000):
      #this function will be based around evalauting the energy of the circuit.
      #it takes in the list of angles from a cirq, nshot is for how many times the cirq should run and it returns an energy score.
      qc = self.create_ansatz(params) #this is used to build a qunatum circuit with the angles from the previous function
      qc.add_register(ClassicalRegister(self.n_qubits)) #this clasical register is a storage unit designed to hold classical bits, which represent binary states or bitstring of (0 and 1s)
      qc.measure_all() #this tells the quantum circuit to measure every qubit so a bitstring is outputted like 0101 0r 10101
      #baicallty what this says is measure all the incomign qubits then place them in the classical register

      sample = Sampler() #what this does is run the circuit and gives us the data
      job = sample.run(qc, shots = n_shots) #what this does is run the ciruit 2000 times to see all of the differnt bitstrings it'll output. so run it 2k times and collect the outcome
      result = job_result() #this gets the data from the run

      counts = result.quasi_dists[0] #this gives you the oputput in dictonioary form. quasi_dists stands for quasi-probability distributions.
      # as an output it could like this [{0: 0.05, 1: 0.10, 2: 0.25, 3: 0.60}] That means Bitstring 000 (0) came up ~5% of the time bitstring 001 (1) ~10% Bitstring 010 (2) ~25% bitstring 011 (3) ~60%
      #overall it basically means get the outcome probabilities from the first circuit run, so I can loop through each bitstring and score it based on how well it folded the protein.
      energies = [] #This is a list to store energy values for each output

      for bitstring, count in counts.itme(): #this loops through each result and its probabilty
        binary_str = format(bitstring, f'0{self.n_qubits}b') #what this is basically doing is taking the integer bitstring and turn it into a nice, full-length binary string (like '0101') that matches the number of qubits we're using.

        #heres an example of what happens If self.n_qubits = 4, and bitstring = 3, then: format(3, '04b') → '0011'

        energy = self.hamiltonian.total_hamiltonian(binary_str) #so with this line, what it's doing is calling the hamitlontian to calcute the energy the bitstring takes. so for exmapl if we have a bitstirng of like 0011, the energy used to make the fold is 2.5. it stakes 2.5 units to hold that shape.

        energies.extend([energy] * int(count * n_shots)) #so this is adding energy values to a list multiple times, depending on how often that folding showed up in the simulation.
        #for example Let’s say this fold showed up 30% of the time (count = 0.3) n_shots = 2000, this means 0.3 * 2000 = 200 times You add [energy] * 300 → so this energy gets added 300 times and so now the list reflects how often each folding happened
        energies = np.array(energies) #what this is doing is that it it converts the list of energies into a numPy array so we can use numPy functions like sort() and mean().
        energies.sort() #it sorts all of the energy values from ascending order from low to high

        cutoff_in = int(len(energies) * self.alpha) #
        if cutoff_idx == 0:
            cutoff_idx = 1  # Always keep at least 1

        # Average the best energies to get our CVaR value
        cvar_energy = np.mean(energies[:cutoff_idx])

        return cvar_energy  # Lower is better
    def optimize(self, initial_params=None, maxiter=100):
        """
        Try different parameter values to find the ones that give the lowest CVaR energy.

        initial_params: Starting guess. If not given, we pick random angles.
        maxiter: How many times to try new parameters.

        Returns the best parameters and their corresponding energy.
        """
        # If we don’t have a starting guess, create random parameters
        if initial_params is None:
            initial_params = np.random.uniform(0, 2 * np.pi, 2 * self.n_qubits)

        # Use a classical algorithm (COBYLA) to minimize the energy
        result = minimize(
            self.evaluate_energy,         # What we're trying to minimize
            initial_params,               # Starting guess
            method='COBYLA',              # Optimization algorithm
            options={'maxiter': maxiter}  # How many steps to take
        )

        return result.x, result.fun  # Return best params and best energy










NameError: name 'List' is not defined